# Load Packages

In [5]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.models.rue.save_load_model import load_model
from src.models.infernoise.tuning import tune_infernoise
from src.models.infernoise.predicting import infernoise_test_predictions
from src.misc import create_folder
from seed_file import seed

# seed = 2023
tuning_seed = 2023
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")

# Seed filepaths
fp_cur_tune_folder = join(fp_tuning, str(tuning_seed))
create_folder(fp_cur_tune_folder)
fp_cur_model_folder = join(fp_models, str(seed))
create_folder(fp_cur_model_folder)
fp_cur_predictions_folder = join(fp_predictions, str(seed))
create_folder(fp_cur_predictions_folder)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [2]:
split_dict = load_split_dict(fp_output_data_folder)

# Tuning

In [4]:
if seed == tuning_seed:
    infernoise_best_hp_dict = {}
    for time_label, target_cols in tqdm(split_dict["target_cols"].items()):
        ae_regressor = load_model(
            name=f"rue_{time_label}", fp_checkpoints=fp_cur_model_folder)
        infernoise_tuning_df = tune_infernoise(
            ae_regressor, stddev_list=[0.00001, 0.00005, 0.0001, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5], 
            valid_df=split_dict["valid_df"], inputs=split_dict["feat_cols"], outputs=target_cols, 
            seed=seed, T=10, regressor_label="_"+time_label
        )
        display(infernoise_tuning_df)
        infernoise_tuning_df.to_csv(join(fp_cur_tune_folder, f"tuning_infernoise_{time_label}.csv"))
        infernoise_best_hp_dict[time_label] = infernoise_tuning_df.iloc[infernoise_tuning_df["loss"].argmin(), 0]
    joblib.dump(infernoise_best_hp_dict, join(fp_cur_tune_folder, "all_infernoise_predictor_best_hp.joblib"))
    display(infernoise_best_hp_dict)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

,std,correlation,loss,best_hyperparameter
0,0.00001,0.013303,0.029778,True
1,0.00005,0.013328,0.029778,False
2,0.00010,0.013308,0.029778,False
3,0.00500,0.011156,0.029842,False
4,0.01000,0.008464,0.030023,False
5,0.02000,0.003742,0.030669,False
6,0.05000,-0.005831,0.034601,False
7,0.10000,-0.016790,0.045051,False
8,0.20000,-0.020945,0.071654,False
9,0.50000,-0.011766,0.162222,False


  0%|          | 0/10 [00:00<?, ?it/s]

,std,correlation,loss,best_hyperparameter
0,0.00001,-0.031413,0.033366,True
1,0.00005,-0.031469,0.033366,False
2,0.00010,-0.031479,0.033366,False
3,0.00500,-0.032154,0.033426,False
4,0.01000,-0.033205,0.033571,False
5,0.02000,-0.034385,0.034119,False
6,0.05000,-0.031174,0.037627,False
7,0.10000,-0.027746,0.047583,False
8,0.20000,-0.024957,0.073640,False
9,0.50000,-0.016268,0.163285,False


  0%|          | 0/10 [00:00<?, ?it/s]

,std,correlation,loss,best_hyperparameter
0,0.00001,0.018313,0.036005,True
1,0.00005,0.018297,0.036005,False
2,0.00010,0.018278,0.036005,False
3,0.00500,0.016125,0.036052,False
4,0.01000,0.013671,0.036190,False
5,0.02000,0.009662,0.036738,False
6,0.05000,0.000219,0.040372,False
7,0.10000,-0.013568,0.051059,False
8,0.20000,-0.018230,0.079176,False
9,0.50000,-0.022369,0.176933,False


{'t+1': 1e-05, 't+2': 1e-05, 't+3': 1e-05}

# Prediction

In [ ]:
infernoise_best_hp_dict = joblib.load(
    join(fp_cur_tune_folder, "all_infernoise_predictor_best_hp.joblib"))
for time_label, target_cols in tqdm(split_dict["target_cols"].items()):
    ae_regressor = load_model(
            name=f"rue_{time_label}", fp_checkpoints=fp_cur_model_folder)
    infernoise_valid_df = infernoise_test_predictions(
         ae_regressor, test_df=split_dict["valid_df"], 
         inputs=split_dict["feat_cols"], outputs=target_cols, regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_best_hp_dict[time_label])
    infernoise_test_df = infernoise_test_predictions(
         ae_regressor, test_df=split_dict["test_df"], 
         inputs=split_dict["feat_cols"], outputs=target_cols, regressor_label="_"+time_label, 
        seed=seed, T=10, stddev=infernoise_best_hp_dict[time_label])
    infernoise_valid_df.to_csv(join(fp_cur_predictions_folder, f"infernoise_valid_{time_label[-1]}.csv"))
    infernoise_test_df.to_csv(join(fp_cur_predictions_folder, f"infernoise_test_{time_label[-1]}.csv"))

  0%|          | 0/3 [00:00<?, ?it/s]

: 